In [4]:
import sys
sys.path.append(r'D:\Personal_Python_Projects\venv\venv')

In [5]:
import pandas as pd
import os

In [6]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    print(f'Processing {file_path}: {df.shape}')
    
    unique_seasons = df['Season'].nunique() - 1
    
    if len(df) > 1:
        df = df.tail(1)
    else:
        print(f"DataFrame has less than 2 rows in {file_path}. Skipping processing.")
        return
    
    df.at[df.index[0], 'YEARS'] = unique_seasons
    df.at[df.index[0], 'AGE'] = None
    df['NAT_CH'] = False #Went back to manually edit NAT_CH and AGE, couldn't figure out any code to get what I wanted
    
    unnamed_columns = df.columns[df.columns.str.contains('^Unnamed')]
    df = df.drop(columns=unnamed_columns)
    
    columns_to_drop = ['2P', '2PA', '2P%', 'ORB', 'DRB', 'SOS', 'School', 'Class', 'Conf', 'Career', 'Tm', 'Team', 'Season', 'GS', 'League']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
    
    df['AST'] = pd.to_numeric(df['AST'], errors='coerce')
    df['TOV'] = pd.to_numeric(df['TOV'], errors='coerce')
    df['AST/TO'] = (df['AST'] / df['TOV']).round(2)
    
    df.rename(columns={'TRB': 'RB', 'MP': 'MPG'}, inplace=True)
    
    columns_order = ['AGE', 'YEARS', 'G', 'MPG', 'PTS', 'AST', 'RB', 'BLK', 'STL', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TOV', 'AST/TO', 'PF', 'NAT_CH']
    df = df[[col for col in columns_order if col in df.columns]]
    
    initial_rows = len(df)
    if len(df) > 1:
        df.drop(df.index[:-1], inplace=True)
        rows_dropped = initial_rows - len(df)
        df['YEARS'] = rows_dropped
        
    df.to_csv(file_path, index=False)
    
    print(f'Processed data saved back to {file_path}')

In [7]:
csv_dir = r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners'

In [8]:
for file_name in os.listdir(csv_dir):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_dir, file_name)
        try:
            process_csv(file_path)
        except Exception as e:
            print(f'Error processing {file_path}: {e}')

Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Paolo Banchero.csv: (2, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Paolo Banchero.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Victor Wembanyama.csv: (4, 27)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Victor Wembanyama.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Scottie Barnes.csv: (2, 30)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Scottie Barnes.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Lamelo Ball.csv: (2, 27)
Processed data saved back to D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Lamelo Ball.csv
Processing D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\Winners\Ja Morant.c

In [9]:
winners_df = pd.read_csv(r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\ROTY_Winners_Names.csv', encoding='latin1')

dataframes = []

for index, row in winners_df.iterrows():
    pick = row['Draft Pick']
    name = row['Name']
    
    stats_df = pd.read_csv(f'D:\\Personal_Python_Projects\\venv\\Projects\\NBA_ROTY_Prediction\\Winners\\{name}.csv', encoding='latin1')
    
    stats_df['Name'] = name
    stats_df['Draft Pick'] = pick
    dataframes.append(stats_df)

combined_df = pd.concat(dataframes, ignore_index=True)

columns_order = ['Name', 'AGE', 'Draft Pick', 'YEARS', 'G', 'MPG', 'PTS', 'AST', 'RB', 'BLK', 'STL', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TOV', 'AST/TO', 'PF', 'NAT_CH']
combined_df = combined_df[[col for col in columns_order if col in combined_df.columns]]

combined_df.to_csv(r'D:\Personal_Python_Projects\venv\Projects\NBA_ROTY_Prediction\combined_winners_stats.csv', index=False)

In [10]:
combined_df

,Name,AGE,Draft Pick,YEARS,G,MPG,PTS,AST,RB,BLK,...,3P,3PA,3P%,FT,FTA,FT%,TOV,AST/TO,PF,NAT_CH
0,Victor Wembanyama,NaN,1,3.0,68.00,24.90,14.8,1.6,7.70,2.4,...,1.1,3.90,0.287,3.00,3.7,0.807,2.00,0.80,2.10,False
1,Paolo Banchero,NaN,1,1.0,39.00,33.00,17.2,3.2,7.80,0.9,...,1.1,3.30,0.338,3.50,4.8,0.729,2.40,1.33,1.90,False
2,Scottie Barnes,NaN,4,1.0,24.00,24.80,10.3,4.1,4.00,0.5,...,0.5,1.70,0.275,1.70,2.8,0.621,2.50,1.64,2.20,False
3,LaMelo Ball,NaN,3,1.0,NaN,31.30,17.0,6.8,7.60,0.1,...,1.7,6.70,0.250,2.80,3.9,0.723,2.50,2.72,2.60,False
4,Ja Morant,NaN,2,2.0,65.00,35.30,18.7,8.2,6.10,0.6,...,1.3,3.80,0.343,5.10,6.3,0.810,3.80,2.16,1.70,False
5,Luka Doncic,NaN,6,4.0,94.00,18.70,8.1,3.2,4.10,0.4,...,0.9,2.70,0.320,1.80,2.3,0.766,1.80,1.78,1.50,False
6,Ben Simmons,NaN,1,1.0,33.00,34.90,19.2,4.8,11.80,0.8,...,0.0,0.10,0.333,6.00,9.0,0.670,3.40,1.41,2.80,False
7,Malcolm Brogdon,NaN,36,4.0,136.00,30.60,13.3,2.5,4.10,0.2,...,1.4,3.70,0.365,3.10,3.5,0.876,1.50,1.67,1.80,False
8,Karl-Anthony Towns,NaN,1,1.0,39.00,21.10,10.3,1.1,6.70,2.3,...,0.1,0.20,0.250,2.80,3.4,0.813,1.40,0.79,2.90,False
9,Andrew Wiggins,NaN,1,1.0,35.00,32.80,17.1,1.5,5.90,1.0,...,1.2,3.60,0.341,5.00,6.5,0.775,2.30,0.65,2.70,False


In [ ]:
#Had to manually edit AGE and NAT_CH columns, plus edit lebron and amare stoudemire myself